**To Do**
<s>
- Add parts to Testing PDF Loaders<br>
 <small>(PDF Loaders + llm model version 을 복합적으로 비교할 수 있도록 테스트 만들기. 현재는 llm model 성능만 비교하도록 구성하였음)</small><br>
 </s>

- Make human-made questions for testing<br>
 <small>(모델 성능 비교할 수 있도록 임베딩한 document의 내용을 기반으로 한 questions - ground truth 쌍 제작하기)</small>

- Make Synthetic dataset to make testing more easier<br>
 <small>(LLM 기반으로 questinos - ground truth 쌍 제작하는 RAGAS의 synthetic dataset 제작하여 Test에 사용하기)</small>

- Make visualization plot for comparing performances of models<br>
 <small>(모델 성능 비교 결과 시각화 만들기.. PPT 발표용...)</small>

In [121]:
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [122]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if OPENAI_API_KEY:
    print("API key loaded successfully.")
else:
    raise ValueError("Error loading API key. Check that OPENAI_API_KEY is set inside .env")

API key loaded successfully.


In [123]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def docs_splitter(docs):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
    )
    all_splits = text_splitter.split_documents(docs)
    return all_splits

In [124]:
## make embeddings using pdf loaders below :
from langchain_community.document_loaders import PyPDFLoader
import pymupdf4llm
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from langchain.text_splitter import CharacterTextSplitter
import nest_asyncio
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# the vesion of PDF Loaders
# (1) PyPDFLoader
# (2) PyMyPDF4llm
# (3) LlamaParser

def make_embeddings(loaderType, file_name):
    FILE_PATH = os.path.join(os.getcwd(), file_name)
    loaderType_dict = {1 : "PyPDFLoader", 2 : "PyMyPDF4llm", 3 : "LlamaParser"}
    if loaderType_dict[loaderType] == "PyPDFLoader":
        loader = PyPDFLoader(file_path=file_path)
        docs = loader.load()

    elif loaderType_dict[loaderType] == "PyMyPDF4llm":
        loader = pymupdf4llm.LlamaMarkdownReader()
        docs_llama = loader.load_data(file_path=file_path)
        doc_creator = CharacterTextSplitter()
        docs = doc_creator.create_documents(metadatas=list(map(lambda x : x.metadata, docs_llama)), 
                                    texts=list(map(lambda x : x.text, docs_llama)))
                                    
    elif loaderType_dict[loaderType] == "LlamaParser":
        nest_asyncio.apply()
        parser = LlamaParse(
            result_type="markdown", 
            num_workers=8, 
            verbose=True
        )
        file_extractor = {".pdf": parser}
        docs = SimpleDirectoryReader(
            input_files=[file_path],
            file_extractor=file_extractor,
        ).load_data()
        docs = [doc.to_langchain_format() for doc in documents]
        
    else:
        print("Wrong loader type is entered.")
        return False

    all_splits = docs_splitter(docs)
    try:
        vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
                                    persist_directory="data")
        print("Embeded vector store is succesfully made.")
        return vectorstore
    except:
        print("Embedding is Failed ... ㅠㅠ")
        return False

In [125]:
FILE_PATH = os.path.join(os.getcwd(), "examplefiles/DB Concepts Chapter1.pdf")
make_embeddings(loaderType=1, file_name=FILE_PATH)

Embeded vector store is succesfully made.


In [126]:
class ChatBot_test():
    def __init__(self, chat_llm):
        load_dotenv()

        self.llm_version = chat_llm
        # the version of OpenAI models
        # (1) gpt-3.5-turbo
        # (2) gpt-3.5-turbo-0613
        # (3) gpt-3.5-turbo-16k-0613
        # (4) gpt-3.5-turbo-instruct-0914
        # (5) gpt-4
        # (6) gpt-4o-mini

        CORPUS_PATH = os.path.join(os.getcwd(), "corpus")  
        CHROMA_PATH = os.path.join(os.getcwd(), "data") 

        self.chat_history = [] 
        self.llm = ChatOpenAI(model=self.llm_version, temperature=0, openai_api_key=OPENAI_API_KEY)
        # => if time left, I'll do experiments on which temperature is ideal between 0 and 1

        self.vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=OpenAIEmbeddings())
        self.retriever = self.vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

        self.prompt = PromptTemplate(
            input_variables=["history", "context", "question"],
            template="""
            You are a knowledgeable assistant. Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

            Conversation history:
            {history}

            Context:
            {context}

            Question:
            {question}

            Answer:
        """)

        print("ChatBot initialized successfully!")    

    def format_docs(self, docs):
            """Format the retrieved documents into a single context string for the prompt."""
            return "\n\n".join(doc.page_content for doc in docs)
        
    def format_history(self):
        """Format the chat history into a string for inclusion in the prompt."""
        return "\n".join(
            f"Q: {item['question']}\nA: {item['answer']}" for item in self.chat_history[-3:]
        )

    def answer(self, question):
        """Generate an answer using the RAG chain."""
        rag_chain = (
            {
                "history": RunnableLambda(lambda _: self.format_history()), 
                "context": self.retriever | self.format_docs,
                "question": RunnablePassthrough()
            }
            | self.prompt
            | self.llm
            | RunnableLambda(lambda x: x.content) 
        )

        response = rag_chain.invoke(question)
        self.chat_history.append({"question": question, "answer": response})

        return response

#### Performance Testing Example

In [127]:
bot = ChatBot_test("gpt-3.5-turbo")

ChatBot initialized successfully!


In [128]:
from datasets import Dataset
from ragas import EvaluationDataset

questions = ["What is definition of DBMS?", 
             "Please tell me some representative examples of Database applications.",
             "What kinds of data storage is used in 1950s?",
             "Relational model concept is defined by whom? Please tell me his/her name.",
             "What is DML and DDL?"
            ]
ground_truths = ["A database-management system (DBMS) is a collection of interrelated data and a set of programs to access those data.",
                "Database can be used for Enterprise Information, Banking and Finance, Universities, Airlines and Telecommunication so on.",
                "Magnetic tapes were developed for data storage. Data processing tasks such as payroll were automated, with data stored on tapes.",
                "A landmark paper by Codd [1970] defined the relational model and nonprocedural ways of querying data in the relational model, and relational databases were born.",
                "A data-manipulation language (DML) is a language that enables users to access or manipulate data. And a data-definition language (DDL) is a language for specifying the database schema and as well as other properties of the data."]
answers = []
contexts = []

CHROMA_PATH = os.path.join(os.getcwd(), "data") 

vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Inference
for query in questions:
  answers.append(bot.answer(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "user_input": questions,
    "reference": ground_truths,
    "response": answers,
    "retrieved_contexts": contexts
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
eval_dataset = EvaluationDataset.from_hf_dataset(dataset)

In [ ]:
from ragas import evaluate
from ragas.metrics import answer_relevancy, context_precision, faithfulness, context_recall, answer_correctness
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

result = evaluate(
    eval_dataset, 
    metrics = [answer_relevancy, context_precision, faithfulness, context_recall, answer_correctness], 
    llm = llm_model, 
    embeddings=OpenAIEmbeddings(), 
    raise_exceptions=False
)

result.to_pandas()

Evaluating: 100%|██████████| 10/10 [00:07<00:00,  1.36it/s]


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy
0,What is definition of DBMS?,[CHAPTER1IntroductionAdatabase-management syst...,A database management system (DBMS) is a colle...,A database-management system (DBMS) is a colle...,0.500,0.877195
1,Please tell me some representative examples of...,[of a query.Exercises1.7List four applications...,Some representative examples of Database appli...,Database can be used for Enterprise Informatio...,0.625,0.972210
2,What kinds of data storage is used in 1950s?,[data from tapesand card decks.•Late 1960s and...,"In the 1950s, magnetic tapes were developed fo...",Magnetic tapes were developed for data storage...,1.000,0.857165
3,Relational model concept is defined by whom? P...,[character) may be usedto delimit records. The...,The relational model concept is defined by Edg...,A landmark paper by Codd [1970] defined the re...,0.000,0.913699
4,What is DML and DDL?,"[10Chapter 1Introductiondates. In practice, th...","DML stands for Data Manipulation Language, whi...",A data-manipulation language (DML) is a langua...,1.000,0.920456


In [135]:
result.to_pandas().describe()

,faithfulness,answer_relevancy
count,5.000000,5.000000
mean,0.625000,0.908145
std,0.414578,0.044282
min,0.000000,0.857165
25%,0.500000,0.877195
50%,0.625000,0.913699
75%,1.000000,0.920456
max,1.000000,0.972210
